In [1]:
import pandas as pd
import numpy as np
import os, time,json,sys
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import random,warnings
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import xgboost as xgb

In [2]:
warnings.simplefilter("ignore")
random.seed(42)
np.random.seed(42)
pd.set_option('display.max_columns', None)

In [3]:
train =pd.read_csv("DATA  Scientist Assignment/train.csv")
train_labels = pd.read_csv("DATA  Scientist Assignment/trainLabels.csv")
train_labels.drop("id", axis=1,inplace=True)
test_data = pd.read_csv("DATA  Scientist Assignment/test.csv")
test_data.columns = train.columns

In [4]:
object_columns = train.select_dtypes(object).columns.tolist()
bool_columns = [i for i in object_columns if train[i][0].isupper()]
bool_columns
train[bool_columns]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,x30,x31,x32,x33,x41,x42,x43,x44,x45,x55,x56,x57,x62,x63,x71,x72,x73,x74,x75,x85,x86,x87,x92,x93,x101,x102,x103,x104,x105,x115,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,YES,NO,YES,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,YES,NO,NO,NO,NO,YES,NO,YES
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NO,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES
2,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,YES,YES,YES,NO,NO,NO,YES,YES,NO,YES,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO
3,YES,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,YES,NO,YES,NO,NO,NO,NO,YES,NO,YES,YES,NO,YES,NO,NO,NO,NO,YES,NO,YES,YES,NO,YES,NO,NO,NO,NO,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,YES
4,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,YES,YES,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES
9995,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,YES,YES,YES,NO,NO,NO,NO,YES,NO,YES,YES,YES,YES,NO,NO,NO,NO,YES,NO,YES,YES,YES,YES,NO,NO,NO,NO,YES,NO,YES,YES,NO,NO,NO,NO,YES,NO,YES
9996,YES,YES,YES,NO,NO,NO,NO,YES,NO,YES,YES,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES
9997,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,NO


In [5]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  #'mean', 'median', or 'most_frequent'
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# bool_transformer = Pipeline(steps=[
#     ('imputer',KNNImputer())
# ])

# Bundle preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, make_column_selector(dtype_include=np.number)),
        ('cat', categorical_transformer, make_column_selector(dtype_include=object))
    ], remainder='passthrough',n_jobs=-1)


In [6]:
X = train
y = train_labels[:9999]

# save data to files

In [7]:
preprocessor.fit(X)

ColumnTransformer(n_jobs=-1, remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001A0F50EB388>),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001A0F50EBD88>)])

In [8]:
preprocessor.feature_names_in_

array(['id', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
       'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19',
       'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28',
       'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37',
       'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46',
       'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55',
       'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64',
       'x65', 'x66', 'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73',
       'x74', 'x75', 'x76', 'x77', 'x78', 'x79', 'x80', 'x81', 'x82',
       'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89', 'x90', 'x91',
       'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100',
       'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x107', 'x108',
       'x109', 'x110', 'x111', 'x112', 'x113', 'x114', 'x115', 'x116',
       'x117', 'x118', 'x119', 'x120', 'x121', 'x122', 'x123', 'x124',
       'x125

In [9]:
# X_df = pd.DataFrame(preprocessor.fit_transform(X).toarray(),columns = preprocessor.feature_names_in_)
# X_test_df = pd.DataFrame(preprocessor.transform(test_data).toarray(),columns = preprocessor.feature_names_in_)

In [10]:
%%time
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

clf = Pipeline(steps=[('pre', preprocessor),
                  ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=250, random_state=42)))])  

clf.fit(X_train,y_train)
print(f"model score: {clf.score(X_val,y_val)}")

from sklearn.metrics import f1_score

preds = clf.predict(X_val)
print(f"f1_score: {f1_score(y_val, preds, average='micro')}")

model score: 0.6845
f1_score: 0.7632972019158054
Wall time: 2min 57s


In [12]:
# xgb_clf = xgb.XGBClassifier(max_depth=5, objective='multi:softprob', n_estimators=1000, 
#                         num_classes=len(y))

In [13]:
%%time
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# clf = Pipeline(steps=[('pre', preprocessor),
#                   ('classifier', MultiOutputClassifier(xgb.XGBClassifier(max_depth=5, objective='multi:softmax', n_estimators=200, 
#                         num_classes=33)))]) 
clf = Pipeline(steps=[('pre', preprocessor),
                  ('classifier', MultiOutputClassifier(xgb.XGBClassifier(n_estimators=200, random_state=42,
                      objective='binary:logistic')))])

clf.fit(X_train,y_train)
print(f"model score: {clf.score(X_val,y_val)}")

from sklearn.metrics import f1_score

preds = clf.predict(X_val)
print(f"f1_score: {f1_score(y_val, preds, average='micro')}")


### do predict on test data.
preds = clf.predict(test_data)
prep_sub_ids = [str(int(s)) + '_y' + str(int(i)) for s in test_data.id for i in range(1, 34)]
flatten_pred_list = preds.flatten()

submission_df = pd.DataFrame([prep_sub_ids,flatten_pred_list]).T
submission_df.columns = ["id","pred"]
# submission_df.head()



model score: 0.781
f1_score: 0.85007006071929
Wall time: 6min 5s


# write submission file

In [ ]:
submission_df.to_csv("xgboost_n_est_200_submission_file.csv",index=False)

# ignore below part 

In [32]:
preds = clf.predict(test_data)
preds,preds.shape

(array([[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0]]),
 (1999, 33))

In [33]:
pd.DataFrame(preds,columns=y_train.columns)

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [37]:
prep_sub_ids = [str(int(s)) + '_y' + str(int(i)) for s in test_data.id for i in range(1, 34)]
print(len(prep_sub_ids))
flatten_pred_list = preds.flatten()

65967
65967


In [40]:
test_prediction_df = pd.DataFrame([prep_sub_ids,flatten_pred_list]).T
test_prediction_df.columns = ["id","pred"]
test_prediction_df.head()

,id,pred
0,1698002_y1,0
1,1698002_y2,0
2,1698002_y3,0
3,1698002_y4,0
4,1698002_y5,0


In [58]:
%%time

## added colsample_bytree and subsample
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# clf = Pipeline(steps=[('pre', preprocessor),
#                   ('classifier', MultiOutputClassifier(xgb.XGBClassifier(max_depth=5, objective='multi:softmax', n_estimators=200, 
#                         num_classes=33)))]) 
clf = Pipeline(steps=[('pre', preprocessor),
                  ('classifier', MultiOutputClassifier(xgb.XGBClassifier(colsample_bytree= 0.85,n_estimators=200,
                                       random_state=42,objective='binary:logistic',subsample=0.9)))])
clf.fit(X_train,y_train)
print(f"model score: {clf.score(X_val,y_val)}")

from sklearn.metrics import f1_score

preds = clf.predict(X_val)
print(f"f1_score: {f1_score(y_val, preds, average='micro')}")

### do predict on test data.
preds = clf.predict(test_data)
prep_sub_ids = [str(int(num)) + '_y' + str(int(i)) for num in test_data.id for i in range(1, 34)]
flatten_pred_list = preds.flatten()

submission_df = pd.DataFrame([prep_sub_ids,flatten_pred_list]).T
submission_df.columns = ["id","pred"]
# submission_df.head()

submission_df.to_csv("xgboost_n_est_200_submission_file.csv")

model score: 0.776
f1_score: 0.8441224585183454
Wall time: 4min 56s


# more testing

In [59]:
%%time

## added colsample_bytree and subsample
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# clf = Pipeline(steps=[('pre', preprocessor),
#                   ('classifier', MultiOutputClassifier(xgb.XGBClassifier(max_depth=5, objective='multi:softmax', n_estimators=200, 
#                         num_classes=33)))]) 
clf = Pipeline(steps=[('pre', preprocessor),
                  ('classifier', MultiOutputClassifier(xgb.XGBClassifier(colsample_bytree= 0.7,n_estimators=250,
                                       random_state=42,objective='binary:logistic',subsample=0.7,n_jobs=-1)))])
clf.fit(X_train,y_train)
print(f"model score: {clf.score(X_val,y_val)}")

from sklearn.metrics import f1_score

preds = clf.predict(X_val)
print(f"f1_score: {f1_score(y_val, preds, average='micro')}")


### do predict on test data.
preds = clf.predict(test_data)
prep_sub_ids = [str(int(num)) + '_y' + str(int(i)) for num in test_data.id for i in range(1, 34)]
flatten_pred_list = preds.flatten()

submission_df = pd.DataFrame([prep_sub_ids,flatten_pred_list]).T
submission_df.columns = ["id","pred"]
# submission_df.head()

submission_df.to_csv("xgboost_n_est_250_submission_file.csv")

model score: 0.766
f1_score: 0.8350563909774437
Wall time: 6min 34s
